In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import missingno as msno
import plotly.express as px # plotting geo data
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.feature_selection import mutual_info_regression
from sklearn.decomposition import PCA
import autofeat
import osmnx as ox
import geopandas as gpd
from tqdm import tqdm

from Own_imputers import PriorityGroupImputer
from OwnFeatEng import feature_engineering, feature_selection, add_geo_features
from OwnDimRed import PCA_num
from PrepData import preprocess
import gc

# Train

## Data

In [2]:
type_map = {
    'energy_label': 'category',
    'postcode': 'category',
    'advertiser' : 'category',
    'province' : 'category', 
    'house_type' : 'category',
    'subtype' : 'category',
    'new_building': 'bool'
}

In [3]:
data = pd.read_csv("train.csv", header=0)
data = data.drop(columns=['is_promoted','sticker','price_drop_date'])
testdata = pd.read_csv("test.csv", header=0)
testdata = testdata.drop(columns=['is_promoted','sticker','price_drop_date'])

X_data, y_data = data.drop(columns='price'), data['price']
# fulldata = pd.concat([X_data,testdata]).reset_index(drop=True)

fullgeofeatures_train = pd.read_parquet('fullgeofeatures.parquet').iloc[:X_data.shape[0]]
geofeatcols = fullgeofeatures_train.columns.to_list()
fullgeofeatures_test = pd.read_parquet('fullgeofeatures.parquet').iloc[X_data.shape[0]:]


X_data = pd.concat([X_data, fullgeofeatures_train], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data,test_size=0.2, shuffle=True, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, shuffle=True, test_size=0.2, random_state=42)

In [4]:
prep = preprocess(type_map=type_map)
X_train = prep.train(X_train).reset_index(drop=True)
X_val = prep.test(X_val).reset_index(drop=True)

# fulldata = prep.train(fulldata).reset_index(drop=True)
gc.collect()

0

In [5]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()

# le = LabelEncoder()
# X_train['house_type'] = le.fit_transform(X_train['house_type'])  
# X_val['house_type'] = le.transform(X_val['house_type'])  

## Feature engineering

### loading geodata

In [6]:
# import osmnx as ox
# import matplotlib.pyplot as plt
# import geopandas as gpd
# from shapely.geometry import Point, box



# # Step 1: Convert to GeoDataFrame
# gdf_houses = gpd.GeoDataFrame(
#     fulldata,
#     geometry=[Point(xy) for xy in zip(fulldata['lon'], fulldata['lat'])],
#     crs="EPSG:4326"
# )

# # Step 2: Project to UTM
# utm_crs = gdf_houses.estimate_utm_crs()
# gdf_houses = gdf_houses.to_crs(utm_crs)

# # Step 3: Define bounding box with buffer

# intersection_gdf = gpd.read_file('intersection_gdf_espg4326.gpkg').to_crs(utm_crs)

# # Step 5: Plot
# fig, ax = plt.subplots(figsize=(10, 10))
# gdf_houses.plot(ax=ax, color='blue', markersize=1, label='Houses')

# intersection_gdf.boundary.plot(
#     ax=ax, color='purple', linewidth=2, label='Belgium + 10km Buffer'
# )
# plt.legend()
# plt.title("Bounding Box vs House Points")
# plt.show()

# intersection_gdf_espg4326 = intersection_gdf.to_crs("EPSG:4326").geometry.iloc[0]

In [7]:
# useamenity = ["restaurant", "fast_food", "cafe", "bar", "pub", 'nightclub' ,'stripclub', 'internet_cafe',
#                 "college", 'kindergarten', 'library', 'research_institute', 'music_school' , 'school', 'university', 'driving_school', 'traffic_park', 'driver_training',
#                 'bus_station', 'taxi',
#                 'bank',
#                 'clinic', 'dentist ' ,'doctors','hospital','nursing_home', 'pharmacy', 'social_facility', 'veterinary',
#                 'arts_centre','brothel','casino','cinema','community_centre','conference_centre','events_venue','exhibition_centre','fountain','gambling','music_venue',
#                 'planetarium','social_centre', 'stage', 'theatre',
#                 'courthouse', 'fire_station' ,'police', 'prison', 'townhall',
#                 'crematorium','funeral_hall','grave_yard','mortuary',
#                 'charging_station', 'fuel']
# area_calc = []
# tag_amenity = {'amenity': True}
# tag_shop = {'shop': True}
# tag_leisure = {'leisure': True}
# tag_tourism = {'tourism': True}
# tag_landuse = {'landuse': True}
# tag_natural = {'natural': True}

# tag_office = {'office': True}
# tag_healthcare = {'healthcare': True}

In [8]:
# gdf_healthcare= ox.features.features_from_polygon(intersection_gdf_espg4326, tags=tag_healthcare) #[['geometry','office']]
# gdf_healthcare[['geometry','healthcare']].to_file("gdf_healthcare.gpkg", driver="GPKG")
# print(gdf_healthcare.head())

In [9]:
# fig, ax = plt.subplots(figsize=(10, 10))
# gdf_tourism.plot(
#     ax=ax, color='purple', markersize=1
# )
# gdf_houses.to_crs(gdf_tourism.crs).plot(ax=ax, color='blue', markersize=1, label='Houses')
# plt.show()

In [10]:
# valid_in_any_group = (
#     gdf_landuse.groupby('amenity')
#     .apply(lambda g: ~g.isna().any(), include_groups=False) 
#     .any(axis=0)                       
# )

### features from geodata


In [11]:
# # X_try  = X_train.iloc[:3000]
# gdf_houses_espg4326 = gpd.GeoDataFrame(
#     fulldata,
#     geometry=[Point(xy) for xy in zip(fulldata['lon'], fulldata['lat'])],
#     crs="EPSG:4326"
# )

In [12]:
gdf_shop = gpd.read_file('gdf_shop.gpkg')
shop_features = gdf_shop.groupby('shop')['geometry'].count()[(gdf_shop.groupby('shop')['geometry'].count()>10)]
# print(shop_features)

gdf_tourism = gpd.read_file('gdf_tourism.gpkg')
tourism_features = gdf_tourism.groupby('tourism')['geometry'].count()[(gdf_tourism.groupby('tourism')['geometry'].count()>10)]
# print(tourism_features)

gdf_natural = gpd.read_file('gdf_natural.gpkg')
natural_features = gdf_natural.groupby('natural')['geometry'].count()[(gdf_natural.groupby('natural')['geometry'].count()>10)]
# print(natural_features)

gdf_landuse = gpd.read_file('gdf_landuse.gpkg')
landuse_features = gdf_landuse.groupby('landuse')['geometry'].count()[(gdf_landuse.groupby('landuse')['geometry'].count()>10)]
# print(landuse_features)

gdf_amenity = gpd.read_file('gdf_amenity.gpkg')
amenity_features = gdf_amenity.groupby('amenity')['geometry'].count()[(gdf_amenity.groupby('amenity')['geometry'].count()>10)]
# print(amenity_features)

gdf_leisure = gpd.read_file('gdf_leisure.gpkg')
leisure_features = gdf_leisure.groupby('leisure')['geometry'].count()[(gdf_leisure.groupby('leisure')['geometry'].count()>10)]
# print(leisure_features)


gdf_office = gpd.read_file('gdf_office.gpkg')
office_features = gdf_office.groupby('office')['geometry'].count()[(gdf_office.groupby('office')['geometry'].count()>10)]
# print(leisure_features)


gdf_healthcare = gpd.read_file('gdf_healthcare.gpkg')
healthcare_features = gdf_healthcare.groupby('healthcare')['geometry'].count()[(gdf_healthcare.groupby('healthcare')['geometry'].count()>10)]
# print(leisure_features)

In [13]:
# tagfeat_dict = {
                # 'amenity': gdf_amenity,
                # 'leisure': gdf_leisure,
                # 'shop': gdf_shop,
                # 'tourism': gdf_tourism,
                # 'natural': gdf_natural,
                # 'landuse': gdf_landuse,
                # 'office' : gdf_office,
                # 'healthcare' : gdf_healthcare
# } 

In [14]:
del gdf_shop, gdf_tourism, gdf_natural, gdf_landuse, gdf_amenity, gdf_leisure, gdf_office, gdf_healthcare
gc.collect()

0

### New own features

In [16]:
# from OwnFeatEng import SpecialTransform

# new_cols = SpecialTransform(X_train).columns.to_list()
# X_train = X_train.join(SpecialTransform(X_train))
# X_val = X_val.join(SpecialTransform(X_val))

In [17]:
# cl_range = list(range(20,101,20)) + list(range(150,301,50))
# feateng = feature_engineering(targets=['area', 'bedrooms','energy_value'],
#                               transform_type=['median'],
#                               comp_type=['diff'],
#                               groups=['province','house_type'],
#                               features=['lat','lon'],
#                               cl_name='clt',
#                               clustering=True, 
#                               cluster_range=cl_range)

# feateng_density = feature_engineering(targets=['area'],
#                               transform_type=['count'],
#                               comp_type=None,
#                               groups=['province','house_type'],
#                               features=['lat','lon'],
#                               cl_name='clt',
#                               clustering=False, 
#                               cluster_range=cl_range)

# X_train_new = feateng.train(X_train)
# X_val_new = feateng.test(X_val)


# X_train_new = feateng_density.train(X_train_new)
# X_val_new = feateng_density.test(X_val_new)

In [18]:
# X_train_new.columns

In [19]:
# features_new = X_train_new.columns.to_list()[24:]
# print(len(features_new))

### Feature selection on correlation

In [20]:
# def getnum(df):
#     logic = (df.dtypes=='float64') | (df.dtypes=='float32') | (df.dtypes=='float16')
#     num_features = df.dtypes[logic].index.to_list()
#     other_features = df.dtypes[~logic].index.to_list()
#     return num_features, other_features

In [21]:
# plt.figure(figsize=(8, 6))
# sns.heatmap(spearman_corr, annot=False, cmap='coolwarm', vmin=-1, vmax=1)
# plt.title('Spearman Correlation Matrix')
# plt.show()

In [22]:
# numcols, othcols = getnum(X_train_new)
# catcols = ['new_building',
#  'advertiser',
#  'subtype',
#  'house_type',
#  'area_miss',
#  'lat_miss',
#  'advertiser_miss',
#  'subtype_miss',
#  'energy_label_miss',
#  'clt20',
#  'clt40',
#  'clt60',
#  'clt80',
#  'clt100',
#  'clt150',
#  'clt200',
#  'clt250',
#  'clt300']

# numcols = numcols + (
# ['miss_tot',
#  'area-clt20-count',
#  'area-clt40-count',
#  'area-clt60-count',
#  'area-clt80-count',
#  'area-clt100-count',
#  'area-clt150-count',
#  'area-clt200-count',
#  'area-clt250-count',
#  'area-clt300-count',
#  'area-province-count',
#  'area-house_type-count' ])

In [23]:
# only_cor = feature_selection()
# selected, _ = only_cor.select(X_train_new[numcols])

In [24]:
# items_to_remove = {'index', 'id', 'is_appartment', 'added_time', 'postcode', 'energy_label', 'province', 'bedrooms_cat', 'lon_miss' ,'energy_value_miss'}
# othcols = [x for x in othcols if x not in items_to_remove]
# features_filt = catcols+selected

### MI scores

In [25]:
# from OwnFeatEng import make_mi_scores, plot_mi_scores
# mi_scores = make_mi_scores(X_train_new_new[features_new],y_train.reset_index(drop=True))
# plot_mi_scores(mi_scores)

### Autofeat

In [26]:
# X_train_new = X_train_new[catcols + numcols]
# X_val_new = X_val_new[catcols + numcols]

In [27]:
# categorical_cols=['house_type', 'lat_miss',  'clt20', 'clt40', 'clt60', 'clt80', 'clt100', 'clt150', 'clt200', 'clt250', 'clt300']
# feateng_cols=['area', 'energy_value', 'bedrooms', 'foto_amount']

# autofeatures = autofeat.AutoFeatRegressor(feateng_cols=feateng_cols, featsel_runs=2)
# X_train_feat = autofeatures.fit_transform(X_train_new[feateng_cols], np.log(y_train.reset_index(drop=True))).drop(columns=feateng_cols)
# X_val_feat = autofeatures.transform(X_val_new[feateng_cols]).drop(columns=feateng_cols)

# X_train_new = X_train_new.join(X_train_feat)
# X_val_new = X_val_new.join(X_val_feat)

### Feature selection


In [28]:
featlist = [
    'amenity*school',
    'amenity*university',
    'amenity*hospital',
    'amenity*pharmacy',
    'amenity*doctors',
    'amenity*clinic',
    'amenity*kindergarten',
    'amenity*park',
    'amenity*playground',
    'amenity*library',
    'amenity*police',
    'amenity*fire_station',
    'amenity*bus_station',
    'amenity*train_station',
    'amenity*subway_entrance',
    'amenity*parking',
    'amenity*charging_station',
    'shop*supermarket',
    'shop*convenience',
    'shop*bakery',
    'shop*butcher',
    'shop*greengrocer',
    'shop*pharmacy',
    'shop*mall',
    'shop*department_store',
    'leisure*park',
    'leisure*golf_course',
    'leisure*swimming_pool',
    'leisure*sports_centre',
    'leisure*stadium',
    'leisure*fitness_centre',
    'natural*water',
    'natural*wood',
    'natural*beach',
    'natural*grassland',
    'natural*wetland',
    'natural*forest',
    'tourism*attraction',
    'tourism*viewpoint',
    'tourism*hotel',
    'tourism*guest_house',
    'tourism*museum',
    'tourism*gallery',
    'tourism*theme_park',
    'office*estate_agent',
    'office*government',
    'office*company',
    'office*university',
    'healthcare*hospital',
    'healthcare*clinic',
    'healthcare*doctor',
    'healthcare*pharmacy',
    'healthcare*dentist',
    'healthcare*laboratory',
    'healthcare*rehabilitation',
    'healthcare*vaccination_centre'
]


In [29]:
top = 100

top = 100/top
shop_features_top = shop_features.sort_values(ascending=False).index.to_list()[:int(np.round(len(shop_features),0))] #/4
tourism_features_top = tourism_features.sort_values(ascending=False).index.to_list()[:int(np.round(len(tourism_features)/top,0))]
landuse_features_top = landuse_features.sort_values(ascending=False).index.to_list()[:int(np.round(len(landuse_features),0))]
natural_features_top = natural_features.sort_values(ascending=False).index.to_list()[:int(np.round(len(natural_features),0))] #/6
amenity_features_top = amenity_features.sort_values(ascending=False).index.to_list()[:int(np.round(len(amenity_features)/top,0))]
leisure_features_top = leisure_features.sort_values(ascending=False).index.to_list()[:int(np.round(len(leisure_features),0))] #/3
office_features_top = office_features.sort_values(ascending=False).index.to_list()[:int(np.round(len(office_features),0))]
healthcare_features_top = healthcare_features.sort_values(ascending=False).index.to_list()[:int(np.round(len(healthcare_features),0))]

shop_features_top = [f'shop*{entry}' for entry in shop_features_top]
tourism_features_top = [f'tourism*{entry}' for entry in tourism_features_top]
landuse_features_top = [f'landuse*{entry}' for entry in landuse_features_top]
natural_features_top = [f'natural*{entry}' for entry in natural_features_top]
amenity_features_top = [f'amenity*{entry}' for entry in amenity_features_top]
leisure_features_top = [f'leisure*{entry}' for entry in leisure_features_top]
office_features_top = [f'office*{entry}' for entry in office_features_top]
healthcare_features_top = [f'healthcare*{entry}' for entry in healthcare_features_top]

#useamenity = ["restaurant", "fast_food", "cafe", "bar", "pub", 'nightclub' ,'stripclub', 'internet_cafe',
                # "college", 'kindergarten', 'library', 'research_institute', 'music_school' , 'school', 'university', 'driving_school', 'traffic_park', 'driver_training',
                # 'bus_station', 'taxi',
                # 'bank',
                # 'clinic', 'dentist ' ,'doctors','hospital','nursing_home', 'pharmacy', 'social_facility', 'veterinary',
                # 'arts_centre','brothel','casino','cinema','community_centre','conference_centre','events_venue','exhibition_centre','fountain','gambling','music_venue',
                # 'planetarium','social_centre', 'stage', 'theatre',
                # 'courthouse', 'fire_station' ,'police', 'prison', 'townhall',
                # 'crematorium','funeral_hall','grave_yard','mortuary',
                # 'charging_station', 'fuel']

#uselanduse = ['landuse*forest', 'landuse*industrial', 'landuse*farmland','landuse*residential', 'landuse*farmyard','landuse*cemetery', 'landuse*commercial',  'landuse*railway','landuse*landfill']


# featlist = shop_features_top + tourism_features_top + natural_features_top + amenity_features_top + leisure_features_top + office_features_top + healthcare_features_top



filterfeats = [ 'mean_',
                'area_tourism',               
                'area_amenity',  
                'count_landuse', #'closest_landuse',
                'area_office', #'office*university', 'office*educational_institution',  'office*healthcare', 'office*religion', 'office*research',
                'area_shop', #'shop*frame',
                'area_healthcare', # 'healthcare*pharmacy', 'healthcare*doctor', 'healthcare*dentist',
            ]
geofeatcols_new = [entry for entry in geofeatcols if not any(w in entry for w in filterfeats)]
threshold = 0.99
geofeatcols_new = X_train[geofeatcols_new].columns[((X_train[geofeatcols_new]==15000) | (X_train[geofeatcols_new]==0)).sum()/len(X_train)<=threshold].to_list()
# geofeatcols_new = [entry for entry in geofeatcols_new if any(w in entry for w in featlist)]

features_init = (['is_appartment',
 'area',
 'bedrooms',
 'new_building',
 'lat',
 'lon',
#  'advertiser',
 'foto_amount',
#  'subtype',
 'energy_value',
#  'province',
 'house_type',
 'area_miss',
 'lat_miss',
 'advertiser_miss',
#  'subtype_miss',
 'energy_value_miss'])

#X_train, X_val, y_train, y_val = train_test_split(train[features],train['price'], shuffle=False, test_size=0.2)
features_model = (features_init
                #+features_new
                +geofeatcols_new
)

## Dimensionality Reduction

### PCA

In [30]:
# pca = PCA_num(n_components=300)
# X_train_pca = pd.DataFrame(pca.fit_transform(X_train[geofeatcols_new]))
# X_val_pca = pd.DataFrame(pca.transform(X_val[geofeatcols_new]))

In [31]:
# X_train_pca[features_init] = X_train[features_init]
# X_val_pca[features_init] = X_val[features_init]

## Model

In [32]:
len(features_model)

5161

### RandomForest


In [33]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error

In [34]:
# rf_reg = RandomForestRegressor(n_estimators=600, random_state=42, max_depth = 15, max_features=300)
# rf_reg.fit(X_train[features_model], y_train.reset_index(drop=True))

In [35]:
# y_pred = rf_reg.predict(X_val[features_model])
# RMSE_score = mean_squared_error(y_val.reset_index(drop=True), y_pred)**(1/2)
# print(RMSE_score)

### KNN

In [36]:
# X_train, y_train = train[['is_appartment','area','lat','lon','foto_amount','energy_value','house_type']].copy(),train['price'].copy()

# X_train_encoded = pd.get_dummies(X_train, columns=['house_type'])

# scaler = StandardScaler()
# X_train_encoded[['area','lat','lon','energy_value','foto_amount']] = scaler.fit_transform(X_train_encoded[['area','lat','lon','energy_value','foto_amount']])

In [37]:
# scores_KNN = []
# for i in range(2,60,2):
#     neigh_model = KNeighborsRegressor(n_neighbors=i, weights='distance')
#     scores_KNN.append(cross_val_score(neigh_model, X_train_encoded, y_train, cv=5, scoring='neg_mean_absolute_percentage_error')*-1)

# scores_KNN = pd.DataFrame(scores_KNN)
# plt.scatter((scores_KNN.index+1)*2, scores_KNN.apply(np.mean, axis=1))

In [38]:
# neigh_model_final = KNeighborsRegressor(n_neighbors=14, weights='distance')
# neigh_model_final.fit(X_train_encoded,y_train)

### XGBoost

In [ ]:
dtrain = xgb.DMatrix(X_train[features_model], label=y_train.reset_index(drop=True), enable_categorical=True)
dval = xgb.DMatrix(X_val[features_model],label=y_val.reset_index(drop=True), enable_categorical=True)

evallist = [(dval, 'val')]
param_xgb = {'max_depth': 8, 'eta': 0.05, 'objective': 'reg:squarederror', 'reg_lambda':5, 'reg_alpha':5, 'eval_metric': 'mape', }
evals_result = {}

xgb_regressor = xgb.train(
    params=param_xgb,
    dtrain=dtrain,
    num_boost_round=2000,
    evals=evallist,
    early_stopping_rounds=20,
    evals_result=evals_result,
    verbose_eval = True
)

[0]	val-rmse:176957.56505
[1]	val-rmse:172589.49644
[2]	val-rmse:168401.93236
[3]	val-rmse:164583.98168
[4]	val-rmse:161029.66077
[5]	val-rmse:157609.27665
[6]	val-rmse:154363.30207
[7]	val-rmse:151318.61911
[8]	val-rmse:148519.90837
[9]	val-rmse:145978.56465
[10]	val-rmse:143443.29170
[11]	val-rmse:141117.20520
[12]	val-rmse:139017.66281
[13]	val-rmse:136979.77032
[14]	val-rmse:135071.17620
[15]	val-rmse:133251.33968
[16]	val-rmse:131532.78230
[17]	val-rmse:129871.68990
[18]	val-rmse:128388.91752
[19]	val-rmse:127012.00526
[20]	val-rmse:125718.57088
[21]	val-rmse:124547.91706
[22]	val-rmse:123395.62285
[23]	val-rmse:122355.25372
[24]	val-rmse:121335.63114
[25]	val-rmse:120290.96896
[26]	val-rmse:119415.24409
[27]	val-rmse:118594.78734
[28]	val-rmse:117775.08632
[29]	val-rmse:117059.39227
[30]	val-rmse:116276.55492
[31]	val-rmse:115644.48803
[32]	val-rmse:114959.29314
[33]	val-rmse:114342.16583
[34]	val-rmse:113691.97750
[35]	val-rmse:113153.32461
[36]	val-rmse:112701.34626
[37]	val-rm

In [40]:
importance = pd.Series(xgb_regressor.get_score(importance_type='total_gain')).sort_values(ascending=False)
featsel = importance.index.to_list()[:150]
featsel = [entry for entry in features_init if not any(w in entry for w in featsel)] + featsel
# featsel = [entry for entry in featsel if not any(w in entry for w in ['shop*frame', 'clock' , 'erotic', 'bench', 'hunting_stand', 'fallen_tree','lounger', 'scuba_diving'])]

In [60]:
# i = 0

dtrain_sel = xgb.DMatrix(X_train[featsel], label=y_train.reset_index(drop=True), enable_categorical=True)
dval_sel = xgb.DMatrix(X_val[featsel],label=y_val.reset_index(drop=True), enable_categorical=True)

evallist_sel = [(dval_sel, 'val')]
param_xgb_sel = {'max_depth': 8, 'eta': 0.05,  'reg_lambda':5, 'reg_alpha':5, 'eval_metric': 'mape', 'tree_method':'hist', 'objective': 'reg:absoluteerror',}
evals_result_sel = {}

xgb_regressor_sel = xgb.train(
    params=param_xgb_sel,
    dtrain=dtrain_sel,
    num_boost_round=5000,
    evals=evallist_sel,
    early_stopping_rounds=20,
    evals_result=evals_result_sel,
    verbose_eval = True
)

[0]	val-mape:0.40669
[1]	val-mape:0.39543
[2]	val-mape:0.38543
[3]	val-mape:0.37570
[4]	val-mape:0.36691
[5]	val-mape:0.35833
[6]	val-mape:0.35004
[7]	val-mape:0.34266
[8]	val-mape:0.33546
[9]	val-mape:0.32853
[10]	val-mape:0.32211
[11]	val-mape:0.31624
[12]	val-mape:0.31086
[13]	val-mape:0.30578
[14]	val-mape:0.30058
[15]	val-mape:0.29587
[16]	val-mape:0.29127
[17]	val-mape:0.28678
[18]	val-mape:0.28313
[19]	val-mape:0.27958
[20]	val-mape:0.27566
[21]	val-mape:0.27253
[22]	val-mape:0.26941
[23]	val-mape:0.26645
[24]	val-mape:0.26367
[25]	val-mape:0.26099
[26]	val-mape:0.25838
[27]	val-mape:0.25608
[28]	val-mape:0.25350
[29]	val-mape:0.25149
[30]	val-mape:0.24957
[31]	val-mape:0.24774
[32]	val-mape:0.24578
[33]	val-mape:0.24397
[34]	val-mape:0.24216
[35]	val-mape:0.24050
[36]	val-mape:0.23884
[37]	val-mape:0.23738
[38]	val-mape:0.23607
[39]	val-mape:0.23470
[40]	val-mape:0.23331
[41]	val-mape:0.23189
[42]	val-mape:0.23072
[43]	val-mape:0.22952
[44]	val-mape:0.22838
[45]	val-mape:0.2272

## Error bars

In [61]:
y_pred_val = xgb_regressor_sel.predict(dval_sel)

In [62]:
q_error = np.quantile(abs(y_pred_val/y_val.reset_index(drop=True)-1),0.8)

# Test

## Data

In [45]:
X_test = prep.test(X_test).reset_index(drop=True)

## KNN

In [48]:
# X_test = test[['is_appartment','area','lat','lon','foto_amount','energy_value','house_type']].copy()

# X_test_encoded = pd.get_dummies(X_test, columns=['house_type'])

# scaler = StandardScaler()
# X_test_encoded[['area','lat','lon','energy_value','foto_amount']] = scaler.fit_transform(X_test_encoded[['area','lat','lon','energy_value','foto_amount']])
# X_test_encoded.isna().apply(sum)
# y_test = test['price'].copy()
# y_pred_test = neigh_model_final.predict(X_test_encoded)

## XGBoost

In [46]:
X_full = pd.concat([X_train[featsel],X_val[featsel]]).reset_index(drop=True)
y_full = pd.concat([y_train,y_val]).reset_index(drop=True)

In [63]:
dtrain_full = xgb.DMatrix(X_full, label=y_full, enable_categorical=True)
param_xgb = {'max_depth': 8, 'eta': 0.05, 'objective': 'reg:absoluteerror', 'reg_lambda':5, 'reg_alpha':5, }

xgb_regressor_full = xgb.train(
    params=param_xgb,
    dtrain=dtrain_full,
    num_boost_round=500,
)

In [64]:
dtest = xgb.DMatrix(X_test[featsel], label=y_test.reset_index(drop=True), enable_categorical=True)

In [65]:
y_pred_test = xgb_regressor_full.predict(dtest)

# Submission metrics

In [50]:
def winkler_score(y_true, lower_bound, upper_bound, alpha):
    """
    Calculates the Winkler Interval Score for a given observation and prediction interval.

    Args:
        y_true (float or np.array): The true observed value(s).
        lower_bound (float or np.array): The lower bound of the prediction interval(s).
        upper_bound (float or np.array): The upper bound of the prediction interval(s).
        alpha (float): The significance level (e.g., 0.1 for a 90% interval).

    Returns:
        float or np.array: The Winkler score(s).
    """
    # Ensure inputs are NumPy arrays for vectorized operations
    y_true = np.asarray(y_true)
    lower_bound = np.asarray(lower_bound)
    upper_bound = np.asarray(upper_bound)

    # Initialize score with the interval width
    score = upper_bound - lower_bound

    # Penalty for observation below lower bound
    penalty_lower = np.where(y_true < lower_bound, (2 / alpha) * (lower_bound - y_true), 0)
    score += penalty_lower

    # Penalty for observation above upper bound
    penalty_upper = np.where(y_true > upper_bound, (2 / alpha) * (y_true - upper_bound), 0)
    score += penalty_upper

    return score


In [66]:
y_pred_final = pd.DataFrame()
# y_pred_final['id']=test['id']
y_pred_final['lower']=y_pred_test*(1-q_error)
y_pred_final['upper']=y_pred_test*(1+q_error)
y_pred_final['pred']=y_pred_test
y_pred_final['score'] = winkler_score(y_test.reset_index(drop=True), y_pred_final['lower'], y_pred_final['upper'], 0.2)

# y_pred_final.to_csv(f"xgb_simple_2.csv", index=False)
print(np.mean(np.abs(y_pred_final['pred']-y_test.reset_index(drop=True))))
print(y_pred_final['score'].mean())

65043.7782910607
336417.56
